In [1]:
import pandas as pd
# import seaborn as sns
import altair as alt
import folium
import matplotlib.pyplot as plt
import numpy as np

In [2]:
alt.themes.enable('dark')
alt.data_transformers.disable_max_rows()
alt.warnings.simplefilter(action='ignore')

In [3]:
EQ_data = pd.read_csv("NE_1950_2009.csv")

In [4]:
EQ_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054 entries, 0 to 1053
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             1054 non-null   object 
 1   latitude         1054 non-null   float64
 2   longitude        1054 non-null   float64
 3   depth            1054 non-null   float64
 4   mag              1054 non-null   float64
 5   magType          1054 non-null   object 
 6   nst              409 non-null    float64
 7   gap              409 non-null    float64
 8   dmin             362 non-null    float64
 9   rms              633 non-null    float64
 10  net              1054 non-null   object 
 11  id               1054 non-null   object 
 12  updated          1054 non-null   object 
 13  place            1054 non-null   object 
 14  type             1054 non-null   object 
 15  horizontalError  395 non-null    float64
 16  depthError       399 non-null    float64
 17  magError      

In [29]:
mask = ((EQ_data['latitude'] <= 45.) 
        & (EQ_data['latitude'] >= 20.) 
        & (EQ_data['longitude'] <= 154.) 
        & (EQ_data['longitude'] >= 122.)
        # & (EQ_data['mag'] >= 7.)
        )
JP_data = EQ_data[mask]


In [30]:
tile_World_Imagery = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
attr_World_Imagery = 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'

tile_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
attr_WorldStreetMap = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

In [31]:
min_lon, max_lon = 122, 154
min_lat, max_lat = 20, 45

In [32]:
m = folium.Map(
    # max_bounds=True,
    tiles=tile_WorldStreetMap, 
    attr=attr_WorldStreetMap,
    zoom_control=False,
    control_scale=True, 
    zoom_start=1,
    # min_lat=20,
    # max_lat=45,
    # min_lon=122,
    # max_lon=154,
    
    )

In [33]:
import folium.plugins
import branca.colormap as cm
import folium.features


In [34]:
colormap = cm.linear.inferno.scale(2.5,7)
colormap.caption = "Magnitude"
m.add_child(colormap)
...

Ellipsis

In [35]:
quaqes = folium.map.FeatureGroup()
for index, row in EQ_data.iterrows():
    quaqes.add_child(
        folium.features.CircleMarker(
            location=[row['latitude'],row['longitude']],
            radius=5,
            color=colormap(row['mag']),
            stroke=False,
            fill=True,
            fill_opacity=0.8,
            opacity=0,
            popup=" year:{}\nmag:{}\nmagType:{}\ndepth:{}".format(row['time'].split('-')[0], row['mag'], row['magType'], row['depth']),
        )
    )

m.add_child(quaqes)
...

Ellipsis

In [36]:
from branca.element import Figure
fig = Figure(width=800, height=500)
fig.add_child(m)